In [46]:
import numpy as np
import pandas as pd

import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

In [47]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
hmtl_data = requests.get(url).text

In [48]:
soup = BeautifulSoup(hmtl_data, "html5lib")

In [49]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df_main=pd.DataFrame(table_contents)
df_main['Borough']=df_main['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [50]:
df_main

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [51]:
df_main.shape

(103, 3)

In [52]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_c1da1834db734fffb427667c65fb50e9 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_c1da1834db734fffb427667c65fb50e9 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_c1da1834db734fffb427667c65fb50e9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='woJBs2g1NwDc8apzkU3rUwe7C4u8Mj3nPwkOQjXt29at',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_c1da1834db734fffb427667c65fb50e9)

body = client_c1da1834db734fffb427667c65fb50e9.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-ttcdwcpmxyglym',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)

df_latlong = df_1.rename(columns={'Postal Code': 'PostalCode'})
df_latlong


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [53]:
df_locations = pd.merge(df_main, df_latlong, on= "PostalCode")
df_locations

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Keep the Borough column as a reference

In [54]:
Borough = df_locations.groupby('Borough').mean().reset_index()
Borough

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654624,-79.384184
2,Downtown Toronto Stn A,43.646435,-79.374846
3,East Toronto,43.671110,-79.325428
4,East Toronto Business,43.662744,-79.321558
5,East York,43.704043,-79.335287
6,East York/East Toronto,43.685347,-79.338106
7,Etobicoke,43.655797,-79.537348
8,Etobicoke Northwest,43.706748,-79.594054
9,Mississauga,43.636966,-79.615819


In [55]:
CLIENT_ID = 'CRLKY2K3AWJVBYIKDTYTFFMV4RJF31GB1N1CRJLJGNTXFJTJ' # your Foursquare ID
CLIENT_SECRET = '23GHEZPNIS1V530AEXOL5RWLFY0AXEGHSE1SSZI5HPSJXVZV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CRLKY2K3AWJVBYIKDTYTFFMV4RJF31GB1N1CRJLJGNTXFJTJ
CLIENT_SECRET:23GHEZPNIS1V530AEXOL5RWLFY0AXEGHSE1SSZI5HPSJXVZV


Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
Toronto_info = getNearbyVenues(names=Borough['Borough'], latitudes=Borough['Latitude'], longitudes=Borough['Longitude'])
Toronto_info

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Toronto,43.701980,-79.398954,Balsamico,43.701505,-79.397162,Italian Restaurant
1,Central Toronto,43.701980,-79.398954,Tabülè,43.700932,-79.397078,Middle Eastern Restaurant
2,Central Toronto,43.701980,-79.398954,Little Sister,43.701552,-79.397163,Indonesian Restaurant
3,Central Toronto,43.701980,-79.398954,La Salumeria,43.701255,-79.397117,Deli / Bodega
4,Central Toronto,43.701980,-79.398954,Five Doors North,43.702236,-79.397526,Italian Restaurant
...,...,...,...,...,...,...,...
338,West Toronto,43.652653,-79.449290,Mister Yummy,43.656793,-79.451724,Korean Restaurant
339,York,43.690797,-79.472633,Caldense Bakery,43.691993,-79.467512,Dessert Shop
340,York,43.690797,-79.472633,Subway,43.690218,-79.474050,Sandwich Place
341,York,43.690797,-79.472633,Dollar Tree,43.690296,-79.474667,Discount Store


Let's check how many venues were returned for each Borough

In [63]:
Toronto_info.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,29,29,29,29,29,29
Downtown Toronto,67,67,67,67,67,67
Downtown Toronto Stn A,99,99,99,99,99,99
East Toronto,25,25,25,25,25,25
East Toronto Business,17,17,17,17,17,17
East York,5,5,5,5,5,5
East York/East Toronto,4,4,4,4,4,4
Etobicoke,2,2,2,2,2,2
Etobicoke Northwest,5,5,5,5,5,5


Let's find out how many unique categories can be curated from all the returned venues

In [65]:
print('There are {} unique categories.'.format(len(Toronto_info['Venue Category'].unique())))

There are 137 unique categories.


Analyzing each Borough with one hot encoding

In [68]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_info[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Borough'] = Toronto_info['Borough'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Borough,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,Auto Workshop,Bagel Shop,Bakery,...,Theater,Tibetan Restaurant,Trail,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
Borough_grouped = Toronto_onehot.groupby('Borough').mean().reset_index()
Borough_grouped

,Borough,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,Auto Workshop,Bagel Shop,Bakery,...,Theater,Tibetan Restaurant,Trail,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.034483,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Downtown Toronto,0.000000,0.000000,0.014925,0.014925,0.000000,0.00,0.000000,0.000000,0.000000,...,0.029851,0.00,0.000000,0.0,0.014925,0.014925,0.014925,0.000000,0.0,0.000000
2,Downtown Toronto Stn A,0.000000,0.010101,0.020202,0.000000,0.000000,0.00,0.000000,0.010101,0.030303,...,0.000000,0.00,0.000000,0.0,0.010101,0.000000,0.000000,0.000000,0.0,0.010101
3,East Toronto,0.000000,0.000000,0.040000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,...,0.000000,0.04,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,East Toronto Business,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.058824
5,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,East York/East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.5,0.000000
8,Etobicoke Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Mississauga,0.071429,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


Creating a function that returns a index of the most common Venue Category.

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_ten = 10
indicators = ['st', 'nd', 'rd']

columns = ['Borough']
for ind in np.arange(num_ten):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Borough_sorted_venues = pd.DataFrame(columns=columns)
Borough_sorted_venues['Borough'] = Borough_grouped['Borough']

for ind in np.arange(Borough_g.shape[0]):
    Borough_sorted_venues.iloc[ind, 1:] = return_most_common_venues(Borough_g.iloc[ind, :], num_ten)

Borough_sorted_venues.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Italian Restaurant,Sushi Restaurant,Restaurant,Coffee Shop,Syrian Restaurant,Indian Restaurant,Ice Cream Shop,Metro Station,Gastropub,Middle Eastern Restaurant
1,Downtown Toronto,Coffee Shop,Clothing Store,Cosmetics Shop,Plaza,Bookstore,Theater,Bubble Tea Shop,Diner,Middle Eastern Restaurant,Seafood Restaurant
2,Downtown Toronto Stn A,Coffee Shop,Seafood Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Restaurant,Beer Bar,Café,Bakery
3,East Toronto,Indian Restaurant,Café,Grocery Store,Pizza Place,Pakistani Restaurant,Snack Place,Shopping Plaza,Brewery,Convenience Store,Tibetan Restaurant
4,East Toronto Business,Yoga Studio,Brewery,Gym / Fitness Center,Light Rail Station,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Pizza Place



Creating the Clustering using Kmeans. The model will run 10 times.

In [94]:
B_data = Borough_grouped.drop('Borough', axis=1)
kmeans = KMeans(n_clusters=4, init='k-means++', n_init=15, random_state=0).fit(B_data)
print(kmeans.labels_[:])

[0 0 0 0 0 2 2 3 0 0 1 0 0 0 0]


Creating a final df with all the info 

In [95]:
Borough_sorted_venues.insert(0, 'Cluster Values', value=kmeans.labels_)
Final_data = pd.merge(Borough, Borough_sorted_venues)
Final_data.head()

,Borough,Latitude,Longitude,Cluster Values,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.701980,-79.398954,0,Italian Restaurant,Sushi Restaurant,Restaurant,Coffee Shop,Syrian Restaurant,Indian Restaurant,Ice Cream Shop,Metro Station,Gastropub,Middle Eastern Restaurant
1,Downtown Toronto,43.654624,-79.384184,0,Coffee Shop,Clothing Store,Cosmetics Shop,Plaza,Bookstore,Theater,Bubble Tea Shop,Diner,Middle Eastern Restaurant,Seafood Restaurant
2,Downtown Toronto Stn A,43.646435,-79.374846,0,Coffee Shop,Seafood Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Restaurant,Beer Bar,Café,Bakery
3,East Toronto,43.671110,-79.325428,0,Indian Restaurant,Café,Grocery Store,Pizza Place,Pakistani Restaurant,Snack Place,Shopping Plaza,Brewery,Convenience Store,Tibetan Restaurant
4,East Toronto Business,43.662744,-79.321558,0,Yoga Studio,Brewery,Gym / Fitness Center,Light Rail Station,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Pizza Place


Creating a map

In [96]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.



In [97]:
Toronto_cluster_map = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Final_data['Latitude'], Final_data['Longitude'], Final_data['Borough'], Final_data['Cluster Values']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Toronto_cluster_map)
       
Toronto_cluster_map